## Write using Delta file format using Trigger Once
Let us go through the details about reading and writing data to target location leveraging Spark Structured Streaming APIs.
* Let us write this data to target location using `delta` file format. However, we will use `trigger(once=True)` to run only once.
* We will also validate by running `%fs` command to see if the parquet files are copied or not.
* The files that are available at source at this time will be picked up automatically.

In [0]:
spark.conf.set('spark.sql.streaming.schemaInference', 'true')

In [0]:
ghactivity_df = spark.readStream.json('/mnt/itv-github-db/streaming/landing/ghactivity')

In [0]:
from pyspark.sql.functions import year, date_format

In [0]:
ghactivity_df = ghactivity_df. \
  withColumn('created_year', year('created_at')). \
  withColumn('created_month', date_format('created_at', 'MM')). \
  withColumn('created_dayofmonth', date_format('created_at', 'dd'))

In [0]:
ghactivity_df. \
  writeStream. \
  partitionBy('created_year', 'created_month', 'created_dayofmonth'). \
  format('delta'). \
  option("checkpointLocation", "/mnt/itv-github-db/streaming/bronze/checkpoint/ghactivity"). \
  option("path", "/mnt/itv-github-db/streaming/bronze/data/ghactivity"). \
  trigger(once=True). \
  start()

Out[5]: <pyspark.sql.streaming.StreamingQuery at 0x7f942ec19f70>

* Validating the checkpoint location. We can see multiple folders. These folders will have all the files that are required for the overhead of the checkpoint.

In [0]:
%fs ls /mnt/itv-github-db/streaming/bronze/checkpoint/ghactivity

path,name,size
dbfs:/mnt/itv-github-db/streaming/bronze/checkpoint/ghactivity/commits/,commits/,0
dbfs:/mnt/itv-github-db/streaming/bronze/checkpoint/ghactivity/metadata,metadata,45
dbfs:/mnt/itv-github-db/streaming/bronze/checkpoint/ghactivity/offsets/,offsets/,0
dbfs:/mnt/itv-github-db/streaming/bronze/checkpoint/ghactivity/sources/,sources/,0


* Validating the data location. We should see the files in this location as we are just copying the files in the parquet file format.

In [0]:
%fs ls /mnt/itv-github-db/streaming/bronze/data/ghactivity/created_year=2021/created_month=01

path,name,size
dbfs:/mnt/itv-github-db/streaming/bronze/data/ghactivity/created_year=2021/created_month=01/created_dayofmonth=13/,created_dayofmonth=13/,0


In [0]:
%fs ls dbfs:/mnt/itv-github-db/streaming/bronze/data/ghactivity/created_year=2021/created_month=01/created_dayofmonth=13/

path,name,size
dbfs:/mnt/itv-github-db/streaming/bronze/data/ghactivity/created_year=2021/created_month=01/created_dayofmonth=13/part-00000-78f3d482-6880-47e4-b1a5-d1e90a5332fe.c000.snappy.parquet,part-00000-78f3d482-6880-47e4-b1a5-d1e90a5332fe.c000.snappy.parquet,156018296
dbfs:/mnt/itv-github-db/streaming/bronze/data/ghactivity/created_year=2021/created_month=01/created_dayofmonth=13/part-00001-96b8161b-bdf3-45ca-a3d4-d18b28187464.c000.snappy.parquet,part-00001-96b8161b-bdf3-45ca-a3d4-d18b28187464.c000.snappy.parquet,161758068
dbfs:/mnt/itv-github-db/streaming/bronze/data/ghactivity/created_year=2021/created_month=01/created_dayofmonth=13/part-00002-bd4a948c-1ce5-4b86-8c3e-b490464411b1.c000.snappy.parquet,part-00002-bd4a948c-1ce5-4b86-8c3e-b490464411b1.c000.snappy.parquet,148942970
dbfs:/mnt/itv-github-db/streaming/bronze/data/ghactivity/created_year=2021/created_month=01/created_dayofmonth=13/part-00003-4b061407-c802-41ae-9723-48689dc8469c.c000.snappy.parquet,part-00003-4b061407-c802-41ae-9723-48689dc8469c.c000.snappy.parquet,149214880
dbfs:/mnt/itv-github-db/streaming/bronze/data/ghactivity/created_year=2021/created_month=01/created_dayofmonth=13/part-00004-c122c98d-09eb-4319-98dc-d709f5d76a29.c000.snappy.parquet,part-00004-c122c98d-09eb-4319-98dc-d709f5d76a29.c000.snappy.parquet,139954602
dbfs:/mnt/itv-github-db/streaming/bronze/data/ghactivity/created_year=2021/created_month=01/created_dayofmonth=13/part-00005-2a8a68ff-a6e4-4a59-81ea-4085a858165c.c000.snappy.parquet,part-00005-2a8a68ff-a6e4-4a59-81ea-4085a858165c.c000.snappy.parquet,130032580
dbfs:/mnt/itv-github-db/streaming/bronze/data/ghactivity/created_year=2021/created_month=01/created_dayofmonth=13/part-00006-d210e2ad-090d-4900-8ce4-e12c124bb6bd.c000.snappy.parquet,part-00006-d210e2ad-090d-4900-8ce4-e12c124bb6bd.c000.snappy.parquet,135389472
dbfs:/mnt/itv-github-db/streaming/bronze/data/ghactivity/created_year=2021/created_month=01/created_dayofmonth=13/part-00007-349a1c34-99fb-4cb4-a3b1-4e6ef8447ccc.c000.snappy.parquet,part-00007-349a1c34-99fb-4cb4-a3b1-4e6ef8447ccc.c000.snappy.parquet,129999359
dbfs:/mnt/itv-github-db/streaming/bronze/data/ghactivity/created_year=2021/created_month=01/created_dayofmonth=13/part-00008-6adb55f0-9567-460c-8715-0355c312df3a.c000.snappy.parquet,part-00008-6adb55f0-9567-460c-8715-0355c312df3a.c000.snappy.parquet,125176974
dbfs:/mnt/itv-github-db/streaming/bronze/data/ghactivity/created_year=2021/created_month=01/created_dayofmonth=13/part-00009-2e3f4d9a-5e92-4bc4-b92c-fb2e39f146e5.c000.snappy.parquet,part-00009-2e3f4d9a-5e92-4bc4-b92c-fb2e39f146e5.c000.snappy.parquet,123876787
